In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df =pd.read_csv("https://bit.ly/seoul-120-text-csv")
df.shape

(2645, 5)

In [3]:
df['문서'] = df['제목'] + df['내용']
df.head(3)

,번호,분류,제목,내용,내용번호,문서
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464,아빠 육아휴직 장려금아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려...
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045,[서울산업진흥원] 서울메이드란?서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하...
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485,(강북구) 정비중강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높...


In [4]:
df = df.query("분류.isin(['행정','경제','복지'])")

In [5]:
label_name = '분류'

In [6]:
feature = df['문서']
target = df[label_name]

In [7]:
# target_onehot = pd.get_dummies(target)
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [8]:
categories_array = np.array(target).reshape(-1, 1)
target_onehot = encoder.fit_transform(categories_array)

In [9]:
target_onehot = target_onehot.toarray()


In [10]:
from sklearn.model_selection import train_test_split

feature_train, feature_test, target_train, target_test = train_test_split(feature,target_onehot, test_size=0.2, random_state=42, stratify=target_onehot)
feature_train.shape, feature_test.shape, target_train.shape, target_test.shape

((1710,), (428,), (1710, 3), (428, 3))

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 10000
oov_tok ='<oov>'
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer

In [12]:
tokenizer.fit_on_texts(feature_train)

In [13]:
word_to_index = tokenizer.word_index
sorted(word_to_index)[:10]

["'",
 "'25시",
 "'경복궁",
 "'관리주체는",
 "'기타운동시설운용업'의",
 "'노원'",
 "'당해",
 "'불허용도'란",
 "'새랑'의",
 "'성공창업"]

In [14]:
# train_sequences = tokenizer.texts_to_sequences(feature_train)
# test_sequences = tokenizer.texts_to_sequences(feature_test)
train_sequences = tokenizer.texts_to_sequences([str(text) for text in feature_train])
test_sequences = tokenizer.texts_to_sequences([str(text) for text in feature_test])

In [15]:
max_length = 500
padding_type = 'post' # 뒤에 채우기

In [16]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
import pandas as pd
from LSTM_model import NeuralNetwork
from run_model import compile_model,predict_model

In [17]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [18]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [19]:
from keras.preprocessing.sequence import pad_sequences
feature_train_sp = pad_sequences(train_sequences,padding=padding_type,maxlen=max_length)
feature_test_sp = pad_sequences(test_sequences,padding=padding_type,maxlen=max_length)
print(feature_train.shape)

(1710,)


In [21]:
class_num = len(target.unique())
data_len = len(feature_train_sp[0])
input_data = feature_train_sp # 여기가 데이터 들어가는 곳

In [23]:
model = compile_model(data_len,class_num)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [26]:
feature_train_sp

array([[6165,  992, 6166, ...,    0,    0,    0],
       [6169, 6170, 2422, ...,    0,    0,    0],
       [2425,  223, 1999, ...,    0,    0,    0],
       ...,
       [   1,   52,    1, ...,    0,    0,    0],
       [   1,    1,  663, ...,    0,    0,    0],
       [6164,    1, 2409, ...,    0,    0,    0]])

In [25]:
feature_train_sp.shape,target_train.shape

((1710, 500), (1710, 3))

In [32]:
train_loader = list(zip(feature_train_sp,target_train))

In [33]:
train_loader

[(array([6165,  992, 6166,  173, 1691, 6165,  992, 6166,  173,    1,    1,
          113,    1, 1692,   32, 1693,    1, 1256,    1,    1, 1694, 6167,
           88,   92, 3047,  476, 1695,  274,  511,  396, 1997,  328, 4143,
         1998, 4143, 1998, 4143, 1998, 1998, 6168, 6168, 1998, 1998,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0

In [31]:
# 학습 루프 실행
num_epochs = 10
train_loader = list(zip(feature_train_sp,target_train))
for epoch in range(num_epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

TypeError: flatten() takes from 0 to 1 positional arguments but 2 were given